In [4]:
import os

In [5]:
%pwd

'e:\\disease\\Disease_classification\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'e:\\disease\\Disease_classification'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [9]:
from CNNclassifier.constant import *
from CNNclassifier.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [11]:
import os
import urllib.request as request
import zipfile
from CNNclassifier import logger
from CNNclassifier.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-06 22:46:40,676:INFO:commonyaml file:config\config.yaml loaded successfully]
[2023-07-06 22:46:40,676:INFO:commonyaml file:params.yaml loaded successfully]
[2023-07-06 22:46:40,676:INFO:commoncreated directory at:artifacts]
[2023-07-06 22:46:40,684:INFO:commoncreated directory at:artifacts/data_ingestion]
[2023-07-06 22:46:56,761:INFO:4037744573artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 25371645
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4fde263a10192b0d88ed4adcbc97ee2f173da504ecc132a1c7ef22948cab88bd"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 67F4:3D70:534E10:617320:64A6F376
Accept-Ranges: bytes
Date: Thu, 06 Jul 2023 17:01:44 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10244-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Tim